
# 📚 Custom RAG for PDFs — Summarization + QA Chatbot (Open‑Source LLM)
*Colab-ready notebook generated on 2025-10-29 08:36.*

**What you get:**
- PDF ingestion & chunking
- Chroma vector store with `sentence-transformers/all-MiniLM-L6-v2`
- Optional reranking with `BGE` (`bge-reranker-base` via `FlagEmbedding`)
- Open‑source LLM (choose one):
  - `mistralai/Mistral-7B-Instruct-v0.2` *(default)*
  - `NousResearch/Meta-Llama-3.1-8B-Instruct` *(works on Colab with 4‑bit)*
- Map‑Reduce summarization of full corpus
- RAG QA chain (retrieval → rerank → grounded answer + citations)
- Simple **Gradio** chatbot UI

> Tip: Start with Mistral 7B in 4‑bit (fast & light), then try Llama 3.1 8B if you have more VRAM.


In [ ]:

import sys, platform, torch
print("Python:", sys.version)
print("Platform:", platform.platform())
print("Torch:", torch.__version__ if torch.cuda.is_available() else "not available")
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


## 1) Install dependencies

In [ ]:

%%capture
!pip -q install --upgrade pip
!pip -q install langchain==0.3.7 langchain-community==0.3.7 langchain-text-splitters==0.3.2
!pip -q install chromadb==0.5.12 sentence-transformers==3.2.1
!pip -q install pypdf==5.0.1 pymupdf==1.24.10
!pip -q install transformers==4.46.1 accelerate==0.34.2 bitsandbytes==0.44.1
!pip -q install FlagEmbedding==1.2.11
!pip -q install gradio==4.44.0
!pip -q install pydantic==2.9.2 pydantic-settings==2.5.2


## 2) Configure models & paths

In [ ]:

from dataclasses import dataclass
from typing import Optional
from pathlib import Path

@dataclass
class RAGConfig:
    # LLM options: 'mistral' or 'llama31'
    llm_choice: str = "mistral"  # "mistral" | "llama31"
    mistral_repo: str = "mistralai/Mistral-7B-Instruct-v0.2"
    llama_repo: str = "NousResearch/Meta-Llama-3.1-8B-Instruct"
    load_4bit: bool = True
    max_new_tokens: int = 512
    temperature: float = 0.2
    top_p: float = 0.9

    embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2"
    use_bge_reranker: bool = True
    bge_reranker: str = "BAAI/bge-reranker-base"

    persist_dir: str = "chroma_store"
    collection_name: str = "pdf_rag"
    chunk_size: int = 1000
    chunk_overlap: int = 200

cfg = RAGConfig()
print(cfg)
Path(cfg.persist_dir).mkdir(exist_ok=True)


## 3) Upload PDFs

In [ ]:

from google.colab import files
from pathlib import Path
import shutil

pdf_dir = Path("pdfs")
pdf_dir.mkdir(exist_ok=True)

print("➡️ Choose your PDF ebooks or documents to ingest...")
uploaded = files.upload()  # opens file picker
for fname, _ in uploaded.items():
    shutil.move(fname, pdf_dir / fname)

list(pdf_dir.glob("*.pdf"))


## 4) Parse PDFs → text

In [ ]:

from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

docs = []
for pdf_path in sorted(pdf_dir.glob("*.pdf")):
    try:
        loader = PyPDFLoader(str(pdf_path))
        pages = loader.load()
        docs.extend(pages)
        print(f"Loaded {pdf_path.name}: {len(pages)} pages")
    except Exception as e:
        print(f"Failed to load {pdf_path.name}: {e}")

print("Total pages:", len(docs))
docs[:2]


## 5) Chunking

In [ ]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=cfg.chunk_size,
    chunk_overlap=cfg.chunk_overlap,
    separators=[ "\n\n", "\n", ". ", "? ", "! ", "; ", " ", ""],
)

chunked_docs = splitter.split_documents(docs)
print("Chunks:", len(chunked_docs))
chunked_docs[:1]


## 6) Build / Load Chroma Vector Store

In [ ]:

from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

emb = HuggingFaceEmbeddings(model_name=cfg.embedding_model)

vectorstore = Chroma(
    collection_name=cfg.collection_name,
    persist_directory=cfg.persist_dir,
    embedding_function=emb,
)
# Add only if empty (avoid duplicates on reruns)
if vectorstore._collection.count() == 0:
    vectorstore.add_documents(chunked_docs)
    vectorstore.persist()
    print("✅ Added and persisted documents to Chroma.")
else:
    print("ℹ️ Using existing Chroma collection; skipping add.")


## 7) (Optional) Reranker — BGE base

In [ ]:

reranker = None
if cfg.use_bge_reranker:
    try:
        from FlagEmbedding import FlagReranker
        reranker = FlagReranker(cfg.bge_reranker, use_fp16=True)
        print("✅ BGE reranker loaded.")
    except Exception as e:
        print("⚠️ Could not load BGE reranker:", e)
        reranker = None


## 8) Load an open‑source LLM (4‑bit)

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

def load_llm(cfg):
    if cfg.llm_choice == "llama31":
        repo = cfg.llama_repo
    else:
        repo = cfg.mistral_repo

    bnb_config = None
    if cfg.load_4bit:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
        )

    tok = AutoTokenizer.from_pretrained(repo, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        repo,
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tok,
        max_new_tokens=cfg.max_new_tokens,
        temperature=cfg.temperature,
        top_p=cfg.top_p,
        do_sample=True,
        pad_token_id=tok.eos_token_id,
    )
    return pipe

llm_pipe = load_llm(cfg)
print("✅ LLM pipeline ready.")


## 9) RAG helpers (retrieve → rerank → answer)

In [ ]:

from typing import List, Dict, Any
import textwrap

def retrieve(query: str, k: int = 6):
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(query)
    if reranker is not None:
        pairs = [[query, d.page_content] for d in docs]
        scores = reranker.compute_score(pairs)
        scored = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
        docs = [d for d, s in scored]
    return docs

def make_context(docs: List, max_chars: int = 4000):
    ctx = ""
    sources = []
    for i, d in enumerate(docs):
        chunk = d.page_content.strip()
        meta = d.metadata.copy()
        src = f"{meta.get('source', 'unknown')}#p{meta.get('page', 'NA')}"
        sources.append(src)
        if len(ctx) + len(chunk) + 100 < max_chars:
            ctx += f"\n[Source {i+1}: {src}]\n{chunk}\n"
        else:
            break
    return ctx.strip(), sources

def chat_llm(prompt: str) -> str:
    out = llm_pipe(prompt)[0]["generated_text"]
    # the pipeline returns input + completion, so trim if needed
    if out.startswith(prompt):
        out = out[len(prompt):]
    return out.strip()

SYSTEM_QA = """You are a precise assistant answering questions grounded ONLY in the provided context.
If the answer cannot be found in the context, say you don't know.
Cite sources like [1], [2] corresponding to the context chunks used.
"""

QA_PROMPT_TMPL = """{system}

Question: {q}

Context:
{ctx}

Answer (with citations):"""

def answer_question(q: str, k: int = 6) -> Dict[str, Any]:
    docs = retrieve(q, k=k)
    ctx, sources = make_context(docs)
    prompt = QA_PROMPT_TMPL.format(system=SYSTEM_QA, q=q, ctx=ctx)
    ans = chat_llm(prompt)
    return {"answer": ans, "sources": sources, "used_k": len(docs)}


## 10) Map‑Reduce corpus summarization

In [ ]:

import math

MAP_PROMPT = """You are helping summarize academic/technical texts.
Write a concise bullet summary (3-6 bullets) of the following passage.
Be faithful to the text; no external info.

PASSAGE:
{passage}
"""

REDUCE_PROMPT = """You will merge multiple bullet lists into a single high-quality summary.
- Keep it concise (6-10 bullets).
- Remove redundancy and boilerplate.
- Preserve key definitions, formulas, and results.
- Add section titles if helpful.

BULLET LISTS:
{bullets}
"""

def map_reduce_summary(batch_size: int = 8, max_docs: int = 128):
    subset = chunked_docs[:max_docs]
    bullets = []
    for i in range(0, len(subset), batch_size):
        batch = subset[i:i+batch_size]
        for d in batch:
            prompt = MAP_PROMPT.format(passage=d.page_content[:3000])
            bullets.append(chat_llm(prompt))
    merged = "\n\n".join(bullets)
    final = chat_llm(REDUCE_PROMPT.format(bullets=merged[:12000]))
    return final

# Example (optional run):
# summary = map_reduce_summary(batch_size=6, max_docs=60)
# print(summary)


## 11) Quick test — ask a question

In [ ]:

q = "What is the central thesis of the first uploaded book?"
res = answer_question(q, k=6)
print("Answer:\n", res["answer"])
print("\nSources:", res["sources"])


## 12) Gradio Chatbot UI

In [ ]:

import gradio as gr

with gr.Blocks(title="PDF RAG Chatbot") as demo:
    gr.Markdown("# 📚 PDF RAG Chatbot — Open‑Source LLM")
    gr.Markdown("Ask questions about your uploaded PDFs. Answers are grounded in the retrieved chunks and include citations.")

    chat = gr.Chatbot(height=400, type="messages")
    q_in = gr.Textbox(label="Your question")
    btn = gr.Button("Ask")
    clear = gr.Button("Clear")

    def respond(history, query):
        res = answer_question(query, k=6)
        reply = res["answer"]
        # Add clickable sources list
        if res["sources"]:
            reply += "\n\n**Sources:** " + ", ".join([f"[{i+1}] {s}" for i, s in enumerate(res["sources"])])
        history = history + [(query, reply)]
        return history, ""

    btn.click(respond, [chat, q_in], [chat, q_in])
    clear.click(lambda: ([], ""), None, [chat, q_in])

demo.launch(share=False)
